# Building a Multi-Agent System with MCP

*Adding controls to the MAS and MCP system*

copyright 2025, Denis Rothman

Making the Agent System More Robust

Chúng ta sẽ thêm vào hệ thống đa tác nhân của mình một số cơ chế kiểm soát quan trọng. Những cải tiến này sẽ biến nguyên mẫu chức năng hiện tại thành một hệ thống đáng tin cậy và thông minh hơn. Chúng ta sẽ tập trung vào việc xử lý các sự cố trong thế giới thực, đảm bảo tính chính xác của dữ liệu và triển khai quy trình kiểm soát chất lượng.

Đầu tiên, chúng ta sẽ tăng cường khả năng chịu lỗi trong giao tiếp của hệ thống với API. Chúng ta sẽ thay thế hàm `call_llm` gốc bằng một hàm mới có tên là `call_llm_robust`. Hàm mới này bổ sung cơ chế thử lại tự động: hệ thống giờ đây sẽ cố gắng gọi API nhiều lần nếu lần gọi đầu tiên thất bại. Điều này giúp bảo vệ hệ thống khỏi các sự cố mạng tạm thời.

Tiếp theo, chúng ta sẽ xác thực toàn bộ các thông điệp (messages). Một hàm mới có tên là `validate_mcp_message` sẽ kiểm tra mọi thông điệp được truyền giữa các tác nhân. Hàm này xác nhận rằng thông điệp được định dạng đúng và chứa đầy đủ toàn bộ thông tin bắt buộc. Việc này ngăn chặn dữ liệu bị lỗi làm gián đoạn quy trình làm việc (workflow) của chúng ta.

Chúng ta cũng sẽ mở rộng đội ngũ tác nhân bằng một tác nhân mới `validator_agent` . Tác nhân này có một nhiệm vụ duy nhất: đóng vai người kiểm tra tính đúng đắn của thông tin (fact checker). Nó so sánh bản nháp do tác nhân viết (writer) tạo ra với bản tóm tắt do tác nhân nghiên cứu (researcher) cung cấp nhằm đảm bảo tính nhất quán về mặt sự kiện. Việc này bổ sung thêm một lớp kiểm soát chất lượng cho hệ thống.

Cuối cùng, chúng ta sẽ xây dựng một quy trình làm việc tự hiệu chỉnh. Chúng ta sẽ nâng cấp bộ điều phối để bao gồm vòng lặp xác thực và hiệu đính. Nếu agent kiểm định phát hiện vấn đề trong bản nháp, thì logic mới sẽ gửi lại nội dung này cho agent soạn thảo kèm theo phản hồi. Agent soạn thảo sau đó sẽ thực hiện hiệu đính. Hệ thống giờ đây có khả năng tự sửa chữa các sai sót của chính mình nhằm tạo ra đầu ra cuối cùng chính xác hơn.

In [1]:
import json
import os
import time
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

# The client will automatically read the OPENAI_API_KEY from your environment.
client = OpenAI()
print("OpenAI client initialized.")

OpenAI client initialized.


In [2]:
#@title 2.Defining the Protocol: The MCP Standard
def create_mcp_message(sender, content, metadata=None):
    """Creates a standardized MCP message."""
    return {
        "protocol_version": "1.0",
        "sender": sender,
        "content": content,
        "metadata": metadata or {}
    }

print("--- Example MCP Message (Our Simplified Version) ---")
example_mcp = create_mcp_message(
    sender="Orchestrator",
    content="Research the benefits of the Mediterranean diet.",
    metadata={"task_id": "T-123", "priority": "high"}
)
print(json.dumps(example_mcp, indent=2))

--- Example MCP Message (Our Simplified Version) ---
{
  "protocol_version": "1.0",
  "sender": "Orchestrator",
  "content": "Research the benefits of the Mediterranean diet.",
  "metadata": {
    "task_id": "T-123",
    "priority": "high"
  }
}


## 3. New: Robust Component Controls

Đây là một phần hoàn toàn mới, giới thiệu các nguyên lý kỹ thuật cốt lõi về khả năng phục hồi (resilience) và độ tin cậy (reliability). Chúng tôi đã bổ sung hai hàm quan trọng tại đây. Hàm `call_llm_robust` tăng cường tính bền vững của hệ thống trước các sự cố mạng bằng cách tích hợp cơ chế tự động thử lại (automatic retry mechanism). Hàm `validate_mcp_message` đóng vai trò như một rào cản kiểm soát thiết yếu, đảm bảo mọi thông điệp được truyền giữa các tác nhân (agents) của chúng ta đều được định dạng đúng, từ đó ngăn ngừa hiện tượng sai lệch dữ liệu (data corruption) trong quy trình làm việc.

In [3]:
# --- Hardening the call_llm Function ---
def call_llm_robust(system_prompt, user_content, retries=3, delay=5):
    """A more robust helper function to call the OpenAI API with retries."""
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model="gpt-5.2",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_content}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"API call failed on attempt {i+1}/{retries}. Error: {e}")
            if i < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("All retries failed.")
                return None

# --- The MCP Validator ---
def validate_mcp_message(message):
    """A simple validator to check the structure of an MCP message."""
    required_keys = ["protocol_version", "sender", "content", "metadata"]
    if not isinstance(message, dict):
        print(f"MCP Validation Failed: Message is not a dictionary.")
        return False
    for key in required_keys:
        if key not in message:
            print(f"MCP Validation Failed: Missing key '{key}'")
            return False
    print(f"MCP message from {message['sender']} validated successfully.")
    return True

## 4. New: Agent Specialization Controls

Phần này mở rộng đội Agents của chúng ta từ hai lên ba. Hai tác nhân ban đầu là `researcher_agent` và `writer_agent` đã được nâng cấp để sử dụng hàm mới `call_llm_robust`, nhờ đó trở nên bền bỉ hơn. Quan trọng hơn cả, chúng ta đã giới thiệu một chuyên gia mới: `validator_agent`. Tác nhân này có nhiệm vụ duy nhất là kiểm tra tính đúng đắn của thông tin, so sánh bản nháp do writer lập ra với bản tóm tắt do researcher cung cấp nhằm đảm bảo tính nhất quán về mặt sự thật, qua đó bổ sung một lớp kiểm soát chất lượng thiết yếu cho hệ thống của chúng ta.

In [4]:
# --- Agent 1: The Researcher ---
def researcher_agent(mcp_input):
    """This agent takes a research topic, finds information, and returns a summary."""
    print("\n[Researcher Agent Activated]")
    simulated_database = {
        "mediterranean diet": "Chế độ ăn Địa Trung Hải giàu trái cây, rau củ, ngũ cốc nguyên hạt, dầu ô liu và cá. Các nghiên cứu cho thấy chế độ ăn này liên quan đến nguy cơ mắc bệnh tim mạch thấp hơn, cải thiện sức khỏe não bộ và kéo dài tuổi thọ. Các thành phần chủ chốt bao gồm chất béo không bão hòa đơn và chất chống oxy hóa."
    }
    research_topic = mcp_input['content']
    research_result = simulated_database.get(research_topic.lower(), "No information found.")
    system_prompt = """
    Bạn là một Research Analyst chuyên nghiệp. Nhiệm vụ của bạn là tổng hợp (synthesize) các thông tin được cung cấp thành 3-4 bullet points súc tích. Hãy tập trung hoàn toàn vào các key findings quan trọng nhất, đảm bảo nội dung cô đọng và có giá trị phân tích cao.
    """
    summary = call_llm_robust(system_prompt, research_result)
    print(f"Research summary created for: '{research_topic}'")
    return create_mcp_message(
        sender="ResearcherAgent",
        content=summary,
        metadata={"source": "Simulated Internal DB"}
    )

In [5]:
# --- Agent 2: The Writer ---
def writer_agent(mcp_input):
    """This agent takes research findings and writes a short blog post."""
    print("\n[Writer Agent Activated]")
    research_summary = mcp_input['content']
    system_prompt = "Bạn là một người viết nội dung lành nghề cho một blog về sức khỏe và chăm sóc bản thân. Giọng văn của bạn hấp dẫn, giàu thông tin và mang tính khuyến khích. Nhiệm vụ của bạn là lấy các điểm nghiên cứu sau đây và viết một bài blog ngắn, lôi cuốn (khoảng 150 từ) với một tiêu đề bắt mắt."
    blog_post = call_llm_robust(system_prompt, research_summary)
    print("Blog post drafted.")
    return create_mcp_message(
        sender="WriterAgent",
        content=blog_post,
        metadata={"word_count": len(blog_post.split())}
    )

In [7]:
# --- Agent 3: The Validator ---
def validator_agent(mcp_input):
    """This agent fact-checks a draft against a source summary."""
    print("\n[Validator Agent Activated]")
    source_summary = mcp_input['content']['summary']
    draft_post = mcp_input['content']['draft']
    system_prompt = """
    Bạn là một meticulous fact-checker (chuyên gia kiểm chứng sự thật tỉ mỉ). 
    Nhiệm vụ của bạn là đối soát để xác định xem 'DRAFT' có factually consistent (nhất quán về mặt sự thật) với 'SOURCE SUMMARY' hay không.

    Tuân thủ nghiêm ngặt các điều kiện phản hồi sau:
    1. Nếu mọi luận điểm trong 'DRAFT' đều có bằng chứng xác thực từ 'SOURCE SUMMARY': Chỉ phản hồi duy nhất từ "pass".
    2. Nếu 'DRAFT' chứa bất kỳ thông tin, chi tiết hoặc khẳng định nào KHÔNG xuất hiện trong 'SOURCE SUMMARY': Phản hồi "fail" kèm theo một câu giải thích ngắn gọn về điểm mâu thuẫn hoặc thông tin dư thừa đó.

    Lưu ý: Không thêm lời chào hay dẫn nhập.
    """
    validation_context = f"SOURCE SUMMARY:\n{source_summary}\n\nDRAFT:\n{draft_post}"
    validation_result = call_llm_robust(system_prompt, validation_context)
    print(f"Validation complete. Result: {validation_result}")
    return create_mcp_message(
        sender="ValidatorAgent",
        content=validation_result
    )

## 5. New: Orchestrator Logic Controls

Ở đây, bộ điều phối ban đầu đơn giản đã được thay thế bằng bộ điều phối cuối cùng (`final_orchestrator`) thông minh hơn nhiều. Phiên bản mới này không còn chỉ là một trình quản lý tác vụ tuyến tính nữa, mà giờ đây bao gồm cả vòng lặp xác thực và hiệu đính. Sau khi tác nhân viết (`writer_agent`) hoàn thành bản nháp, bộ điều phối sẽ giao nhiệm vụ cho tác nhân xác thực (`validator_agent`). Nếu quá trình xác thực thất bại, bộ điều phối sẽ gửi lại bản nháp cho tác nhân viết kèm theo phản hồi từ tác nhân xác thực, từ đó tạo nên một hệ thống tự hiệu chỉnh mạnh mẽ, mô phỏng quy trình biên tập trong thực tiễn.

In [8]:
#@title 5.The Final Orchestrator with Validation Loop
def final_orchestrator(initial_goal):
    """Manages the full multi-agent workflow, including validation and revision."""
    print("="*50)
    print(f"[Orchestrator] Goal Received: '{initial_goal}'")
    print("="*50)

    # --- Step 1: Research ---
    print("\n[Orchestrator] Task 1: Research. Delegating to Researcher Agent.")
    research_topic = "Mediterranean Diet"
    mcp_to_researcher = create_mcp_message(sender="Orchestrator", content=research_topic)
    mcp_from_researcher = researcher_agent(mcp_to_researcher)

    if not validate_mcp_message(mcp_from_researcher) or not mcp_from_researcher['content']:
        print("Workflow failed due to invalid or empty message from Researcher.")
        return

    research_summary = mcp_from_researcher['content']
    print("\n[Orchestrator] Research complete.")

    # --- Step 2 & 3: Iterative Writing and Validation Loop ---
    final_output = "Could not produce a validated article."
    max_revisions = 2
    for i in range(max_revisions):
        print(f"\n[Orchestrator] Writing Attempt {i+1}/{max_revisions}")

        writer_context = research_summary
        if i > 0:
            writer_context += f"\n\nPlease revise the previous draft based on this feedback: {validation_result}"

        mcp_to_writer = create_mcp_message(sender="Orchestrator", content=writer_context)
        mcp_from_writer = writer_agent(mcp_to_writer)

        if not validate_mcp_message(mcp_from_writer) or not mcp_from_writer['content']:
            print("Aborting revision loop due to invalid message from Writer.")
            break
        draft_post = mcp_from_writer['content']

        # --- Validation Step ---
        print("\n[Orchestrator] Draft received. Delegating to Validator Agent.")
        validation_content = {"summary": research_summary, "draft": draft_post}
        mcp_to_validator = create_mcp_message(sender="Orchestrator", content=validation_content)
        mcp_from_validator = validator_agent(mcp_to_validator)

        if not validate_mcp_message(mcp_from_validator) or not mcp_from_validator['content']:
            print("Aborting revision loop due to invalid message from Validator.")
            break
        validation_result = mcp_from_validator['content']

        if "pass" in validation_result.lower():
            print("\n[Orchestrator] Validation PASSED. Finalizing content.")
            final_output = draft_post
            break
        else:
            print(f"\n[Orchestrator] Validation FAILED. Feedback: {validation_result}")
            if i < max_revisions - 1:
                print("Requesting revision.")
            else:
                print("Max revisions reached. Workflow failed.")

    # --- Step 4: Final Presentation ---
    print("\n" + "="*50)
    print("[Orchestrator] Workflow Complete. Final Output:")
    print("="*50)
    print(final_output)

## 6. New:Final System Execution Controls

Khối mã cuối cùng này giờ đây chạy toàn bộ hệ thống đã được nâng cấp. Thay vì gọi phiên bản nguyên mẫu đơn giản, nó thực thi <code>final_orchestrator</code>. Điều này cho phép người đọc quan sát toàn bộ quy trình làm việc mạnh mẽ trong hành động, bao gồm cả các bước xác thực và hiệu đính mới, từ đó minh họa đầy đủ sức mạnh của kiến trúc đa tác nhân (multi-agent) có khả năng phục hồi mà chúng ta đã xây dựng.


In [9]:
#@title 6.Run the Final, Robust System
user_goal = "Create a blog post about the benefits of the Mediterranean diet."
final_orchestrator(user_goal)

[Orchestrator] Goal Received: 'Create a blog post about the benefits of the Mediterranean diet.'

[Orchestrator] Task 1: Research. Delegating to Researcher Agent.

[Researcher Agent Activated]
Research summary created for: 'Mediterranean Diet'
MCP message from ResearcherAgent validated successfully.

[Orchestrator] Research complete.

[Orchestrator] Writing Attempt 1/2

[Writer Agent Activated]
Blog post drafted.
MCP message from WriterAgent validated successfully.

[Orchestrator] Draft received. Delegating to Validator Agent.

[Validator Agent Activated]
Validation complete. Result: fail: DRAFT thêm khẳng định không có trong SOURCE SUMMARY như “dễ áp dụng”, “ngon miệng”, “tim khỏe, não sáng”, và khuyến nghị cụ thể “ăn cá vài bữa mỗi tuần/ thêm một đĩa rau/ đổi sang dầu ô liu”.
MCP message from ValidatorAgent validated successfully.

[Orchestrator] Validation FAILED. Feedback: fail: DRAFT thêm khẳng định không có trong SOURCE SUMMARY như “dễ áp dụng”, “ngon miệng”, “tim khỏe, não sáng”